In [2]:
# (Khuyên dùng venv/conda)
%pip install -U ultralytics

  Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Remember restart kernal after install
# Using numpy version 1.26.4 to running RT_DERT
!pip install -U "numpy<2" --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 118.3 MB/s eta 0:00:0000:010:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [3]:
import sys, torch, ultralytics, subprocess, numpy as np, cv2, matplotlib, yaml, requests, polars
import numpy as np

subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "--force-reinstall", "numpy<2"])

print("🔧 Environment versions check\n")
print(f"Python exec path : {sys.executable}")
print(f"Python version   : {sys.version.split()[0]}")
print(f"Ultralytics      : {ultralytics.__version__}")
print(f"Torch            : {torch.__version__} | CUDA: {torch.version.cuda} | cuda_available: {torch.cuda.is_available()}")
print(f"NumPy            : {np.__version__}")
print(f"OpenCV (cv2)     : {cv2.__version__}")
print(f"Matplotlib       : {matplotlib.__version__}")
print(f"PyYAML           : {yaml.__version__}")
print(f"Requests         : {requests.__version__}")
print(f"Polars           : {polars.__version__}")



  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


🔧 Environment versions check

Python exec path : /bin/python3
Python version   : 3.12.3
Ultralytics      : 8.3.223
Torch            : 2.9.0+cu128 | CUDA: 12.8 | cuda_available: True
NumPy            : 1.26.4
OpenCV (cv2)     : 4.12.0
Matplotlib       : 3.10.7
PyYAML           : 6.0.2
Requests         : 2.32.5
Polars           : 1.35.1


In [4]:
import os, glob

data_root = r"/workspace/bdd100k/BDD100k"
labels_dir = os.path.join(data_root, "labels")
for split in ["train", "val"]:
    files = glob.glob(os.path.join(labels_dir, split, "**", "*.txt"), recursive=True)
    non_empty = sum(os.path.getsize(p) > 0 for p in files)
    print(f"{split:<5} | txt: {len(files):>6} | non_empty: {non_empty:>6}")


train | txt:  69863 | non_empty:  69863
val   | txt:  10000 | non_empty:  10000


### Training

In [6]:
WIN_DATA_ROOT = r"/workspace/bdd100k/BDD100k"
WIN_YAML_PATH = r"/workspace/bdd100k_dert.yaml"
RUNS_ROOT     = r"/workspace/runs/detect_dert"

# Tạo file YAML (nếu bạn đã có sẵn file đúng nội dung thì có thể bỏ qua cell này)
yaml_text = f"""path: {WIN_DATA_ROOT}
train: images/train
val: images/val
names:
  0: person
  1: rider
  2: car
  3: truck
  4: bus
  5: train
  6: motor
  7: bike
  8: traffic light
  9: traffic sign
"""

import os
os.makedirs(os.path.dirname(WIN_YAML_PATH), exist_ok=True)
with open(WIN_YAML_PATH, "w", encoding="utf-8") as f:
    f.write(yaml_text)

print("YAML saved to:", WIN_YAML_PATH)
print("Data root     :", WIN_DATA_ROOT)
print("Runs root     :", RUNS_ROOT)


YAML saved to: /workspace/bdd100k_dert.yaml
Data root     : /workspace/bdd100k/BDD100k
Runs root     : /workspace/runs/detect_dert


In [7]:
from ultralytics import YOLO

device = 0 if torch.cuda.is_available() else "cpu"
print("CUDA available:", torch.cuda.is_available())

model = YOLO("rtdetr-l.pt")  # tải trọng số COCO sẵn có
results = model.train(
    data=WIN_YAML_PATH,
    epochs=50,            # 50
    imgsz=640,
    batch=16,             # 64
    device=device,           
    workers=4,           # 4
    cache=True,         # True
    close_mosaic=10,
    save_period=5,
    deterministic=False,
    amp = True,         # True
    # pretrained=True,
    project=RUNS_ROOT,   # thư mục lưu runs
    patience=0,  
    name="bdd100k_rtdetr_l"
)

print("Done training. Best model should be at:")
print(os.path.join(RUNS_ROOT, "bdd100k_rtdetr_l", "weights", "best.pt"))


CUDA available: True
Ultralytics 8.3.223 🚀 Python-3.12.3 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5090, 32120MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/workspace/bdd100k_dert.yaml, degrees=0.0, deterministic=False, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=rtdetr-l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=bdd100k_rtdetr_l6, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, pati

Overriding model.yaml nc=80 with nc=10
WARNING ⚠️ no model scale passed. Assuming scale='l'.

                   from  n    params  module                                       arguments                     
  0                  -1  1     25248  ultralytics.nn.modules.block.HGStem          [3, 32, 48]                   
  1                  -1  6    155072  ultralytics.nn.modules.block.HGBlock         [48, 48, 128, 3, 6]           
  2                  -1  1      1408  ultralytics.nn.modules.conv.DWConv           [128, 128, 3, 2, 1, False]    
  3                  -1  6    839296  ultralytics.nn.modules.block.HGBlock         [128, 96, 512, 3, 6]          
  4                  -1  1      5632  ultralytics.nn.modules.conv.DWConv           [512, 512, 3, 2, 1, False]    
  5                  -1  6   1695360  ultralytics.nn.modules.block.HGBlock         [512, 192, 1024, 5, 6, True, False]
  6                  -1  6   2055808  ultralytics.nn.modules.block.HGBlock         [1024, 192, 1024, 5,